In [1]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Drop the non-beneficial ID columns, 'EIN'.
application_df = application_df.drop(['EIN'], axis=1)

# Determine the number of unique values in each column.
unique_counts = application_df.nunique()

# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Choose a cutoff value and create a list of application types to be replaced
cutoff = 500
application_types_to_replace = list(application_type_counts[application_type_counts < cutoff].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Choose a cutoff value and create a list of classifications to be replaced
cutoff_cls = 100
classifications_to_replace = list(classification_counts[classification_counts < cutoff_cls].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Binning for the 'NAME' column
name_counts = application_df['NAME'].value_counts()
cutoff_name = 100
names_to_replace = list(name_counts[name_counts < cutoff_name].index)

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name, "Other")

# Convert categorical data to numeric with pd.get_dummies
application_df_encoded = pd.get_dummies(application_df)

# Drop the 'NAME' column from X as it has been encoded and 'IS_SUCCESSFUL' column is the target
X = application_df_encoded.drop(['IS_SUCCESSFUL'], axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Experiment with increasing the number of neurons in the first hidden layer
nn.add(tf.keras.layers.Dense(units=64, input_dim=len(X.columns), activation=tf.keras.activations.relu))

# Add another hidden layer with more neurons
nn.add(tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid))

# Check the structure of the model
nn.summary()

# Compile the model with a custom learning rate
nn.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

# Experiment with more epochs for training
nn.fit(X_train_scaled, y_train, epochs=100, batch_size=64)  # Adjust batch size as needed

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                5184      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 7297 (28.50 KB)
Trainable params: 7297 (28.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100


402/402 [==============================] - 2s 2ms/step - loss: 0.5171 - accuracy: 0.7368
Epoch 2/100
402/402 [==============================] - 1s 2ms/step - loss: 0.4937 - accuracy: 0.7554
Epoch 3/100
402/402 [==============================] - 1s 2ms/step - loss: 0.489

In [2]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

f:\Users\Nathan Rosenberg\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
